In [5]:
print('Hello')

Hello


In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
GEMINI_API_KEY=os.getenv("GEMINI_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

In [9]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001" , google_api_key=GEMINI_API_KEY )

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                            google_api_key=GEMINI_API_KEY , 
                            temperature=0.7)

e:\ML\langGraph_Tutorials\l_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

The question "What is the capital of sl" is ambiguous because "sl" could refer to several places. Here are the most likely options and their capitals:

*   **Sierra Leone (SL)**: The capital is **Freetown**.
*   **Sint Maarten (SL)**: The capital is **Philipsburg**.
*   **Sri Lanka (SL)**: The administrative capital is **Sri Jayawardenepura Kotte**, while the commercial capital is **Colombo**.
*   **Slovenia (SL)**: The capital is **Ljubljana**.
*   **Slovakia (SL)**: The capital is **Bratislava**.
goodbye take care yourself


In [13]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [14]:
store={}

In [15]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [16]:
config = {"configurable": {"session_id": "firstchat"}}

In [17]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [18]:
model_with_memory.invoke(("Hi! I'm Rovidu"),config=config).content

"Hi Rovidu! It's nice to meet you. How can I help you today?"

In [19]:
model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Your name is Rovidu. You told me that in your first message!'

In [ ]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

### Reading the txt files from source directory

loader = DirectoryLoader('E:/ML/langGraph_Tutorials/data/', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001" , google_api_key=GEMINI_API_KEY )

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [31]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [32]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [33]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

Based on the provided context, here are three important points about Llama 3:

1.  It has an 8B parameter version.
2.  It was released in April 2024.
3.  It is used in some services.


Agents

In [34]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [35]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=100)

In [36]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)


In [37]:
tool.name

'wikipedia'

In [38]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


In [44]:
from langchain_core.pydantic_v1 import BaseModel, Field

In [45]:

class WikiInputs(BaseModel):
    query: str = Field(description="query to look up in Wikipedia, should be 3 or less words")

In [47]:
tool= WikipediaQueryRun(
    name="wiki-tool",
    description="look up things in wikipedia",
    args_schema=WikiInputs,
    api_wrapper=api_wrapper,
    return_direct=True,
)

ValidationError: 1 validation error for WikipediaQueryRun
args_schema
  Input should be a subclass of BaseModel [type=is_subclass_of, input_value=<class '__main__.WikiInputs'>, input_type=ModelMetaclass]
    For further information visit https://errors.pydantic.dev/2.11/v/is_subclass_of

Custom AGENT

In [48]:
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from pydantic import BaseModel

e:\ML\langGraph_Tutorials\l_venv\Lib\site-packages\IPython\core\interactiveshell.py:3670: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [49]:
@tool
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

In [50]:
print(search.name)
print(search.description)
print(search.args)

search
Look up things online.
{'query': {'title': 'Query', 'type': 'string'}}


In [51]:
@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [52]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [53]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

In [54]:
@tool("search-tool", args_schema=SearchInput, return_direct=True)
def search(query: str) -> str:
    """Look up things online."""
    return "LangChain"

In [55]:
print(search.name)
print(search.description)
print(search.args)
print(search.return_direct)

search-tool
Look up things online.
{'query': {'title': 'Query', 'type': 'string'}}
True


e:\ML\langGraph_Tutorials\l_venv\Lib\site-packages\pydantic\json_schema.py:2324: PydanticJsonSchemaWarning: Default value default=PydanticUndefined description='should be a search query' extra={} is not JSON serializable; excluding default from JSON schema [non-serializable-default]
  warnings.warn(message, PydanticJsonSchemaWarning)


In [56]:
from typing import Optional, Type

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

In [57]:
class SearchInput(BaseModel):
    query: str = Field(description="should be a search query")

In [58]:
class CalculatorInput(BaseModel):
    a: int = Field(description="first number")
    b: int = Field(description="second number")

In [59]:
from langchain.agents import tool
@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

In [60]:
get_word_length.invoke("abc")

3

In [61]:
tools = [get_word_length]

In [63]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [64]:
llm_with_tools = llm.bind_tools(tools)

In [65]:

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

In [66]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [67]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [68]:
list(agent_executor.stream({"input": "How many letters in the word eudca"}))



> Entering new None chain...

Invoking: `get_word_length` with `{'word': 'eudca'}`


5There are 5 letters in the word eudca.

> Finished chain.


[{'actions': [ToolAgentAction(tool='get_word_length', tool_input={'word': 'eudca'}, log="\nInvoking: `get_word_length` with `{'word': 'eudca'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'name': 'get_word_length', 'arguments': '{"word": "eudca"}'}}, response_metadata={'finish_reason': 'STOP', 'safety_ratings': []}, id='run-279b2498-3258-46e9-953c-449f69ebafdf', tool_calls=[{'name': 'get_word_length', 'args': {'word': 'eudca'}, 'id': '9ebcd6da-b362-4e81-aea7-61eb96a684f3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 38, 'output_tokens': 9, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}}, tool_call_chunks=[{'name': 'get_word_length', 'args': '{"word": "eudca"}', 'id': '9ebcd6da-b362-4e81-aea7-61eb96a684f3', 'index': None, 'type': 'tool_call_chunk'}])], tool_call_id='9ebcd6da-b362-4e81-aea7-61eb96a684f3')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'function_call': {'name': 'get_word_length', 'arguments'